In [ ]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [ ]:
def weighted_mean(x, w):
    return sum([x[i]*w[i] for i in range(len(x))])/sum(w)

In [ ]:
batter23 = pd.read_csv('batter_data_23NL.csv')
pitcher23 = pd.read_csv('pitcher_data_23NL.csv')
versus23 = pd.read_csv('versus_data_23NL.csv')
sd_arith23 = torch.load("ckpt/vs_weighted_mean_23ML_3000.pt")
w_arith23 = sd_arith23["weight"].item()

In [ ]:
adjusted_arith_batter23 = []
adjusted_arith_pitcher23 = []
for i in range(len(batter23)):
    pitcher_total = [pitcher23["total"][j] for j in range(len(pitcher23))]
    versus_avg = [weighted_mean([sd_arith23["v1"][i], sd_arith23["v2"][j]], [w_arith23, 1-w_arith23]) for j in range(len(pitcher23))]
    adjusted_arith_batter23.append(weighted_mean(versus_avg, pitcher_total))
for j in range(len(pitcher23)):
    batter_total = [batter23["total"][i] for i in range(len(batter23))]
    versus_avg = [weighted_mean([sd_arith23["v1"][i], sd_arith23["v2"][j]], [w_arith23, 1-w_arith23]) for i in range(len(batter23))]
    adjusted_arith_pitcher23.append(weighted_mean(versus_avg, batter_total))

In [ ]:
batter23["adjusted_arith"] = adjusted_arith_batter23
pitcher23["adjusted_arith"] = adjusted_arith_pitcher23

In [ ]:
batter23_over150 = [batter23["batter"][i] for i in range(len(batter23)) if batter23["total"][i]>=150]
batter23_over150_index = [i for i in range(len(batter23)) if batter23["total"][i]>=150]
pitcher23_over150 = [pitcher23["pitcher"][i] for i in range(len(pitcher23)) if pitcher23["total"][i]>=150]
pitcher23_over150_index = [i for i in range(len(pitcher23)) if pitcher23["total"][i]>=150]

In [ ]:
t = [batter23["total"][i] for i in batter23_over150_index]
x = [batter23["obp"][i] for i in batter23_over150_index]
z = [sd_arith23["v1"][i] for i in batter23_over150_index]
z2 = [batter23["adjusted_arith"][i] for i in batter23_over150_index]

In [ ]:
gap = [[batter23_over150[i],z2[i], batter23_over150_index[i],z[i]] for i in range(len(x))]
gap.sort(key=lambda x:-x[1])
print(gap[:10])
gap.sort(key=lambda x:x[1])
print(gap[:10])

In [ ]:
batter_data_seperated_by_total50 = {}
for i in range(len(t)):
    b = (t[i]-50)//100
    if(b not in batter_data_seperated_by_total50):
        batter_data_seperated_by_total50[b] = []
    batter_data_seperated_by_total50[b].append(float(gap[i][1]))

plt.clf()
fig, ax = plt.subplots()
ax.boxplot([batter_data_seperated_by_total50[i] for i in range(6)])
ax.set_xticklabels([str(i*100+50)+"~"+str(i*100+149) for i in range(6)])
ax.set_xlabel("AB")
ax.set_ylabel("Adjusted AVG - Original AVG")

plt.show()

for i in range(6):
    pd.DataFrame(batter_data_seperated_by_total50[i]).describe()

In [ ]:
gap = [[pitcher23_over150[i],z2[i], pitcher23_over150_index[i],z[i]] for i in range(len(x))]
gap.sort(key=lambda x:-x[1])
print(gap[:10])
gap.sort(key=lambda x:x[1])
print(gap[:10])

In [ ]:
pitcher_data_seperated_by_total50 = {}
for i in range(len(t)):
    b = (t[i]-50)//100
    if(b not in pitcher_data_seperated_by_total50):
        pitcher_data_seperated_by_total50[b] = []
    pitcher_data_seperated_by_total50[b].append(float(gap[i][1]))
    
plt.clf()
fig, ax = plt.subplots()
ax.boxplot([pitcher_data_seperated_by_total50[i] for i in range(8)])
ax.set_xticklabels([str(100*i+50)+"~"+str(i*100+149) for i in range(8)])
ax.set_xlabel("AB")
ax.set_ylabel("Adjusted AVG - Original AVG")
plt.show()

for i in range(8):
    pd.DataFrame(pitcher_data_seperated_by_total50[i]).describe()

In [ ]:
lowest_batter = 592273
highest_batter = 656976
lowest_against = []
highest_against = []

pitcher_to_obp = {pitcher23["pitcher"][i]:pitcher23["obp"][i] for i in range(len(pitcher23))}

for i in range(len(versus23)):
    if(versus23["batter"][i]==lowest_batter):
        for _ in range(versus23["total"][i]):
            lowest_against.append(versus23["pitcher"][i])
    if(versus23["batter"][i]==highest_batter):
        for _ in range(versus23["total"][i]):
            highest_against.append(versus23["pitcher"][i])
            
lowest_against = [pitcher_to_obp[p] for p in lowest_against]
highest_against = [pitcher_to_obp[p] for p in highest_against]

lowest_against = pd.DataFrame(lowest_against)
highest_against = pd.DataFrame(highest_against)

print(lowest_against.describe())
print(highest_against.describe())

plt.clf()
plt.hist(lowest_against, bins=20)
plt.xlabel("pitcher's AVG")
plt.xlim(0.1, 0.4)
plt.ylabel("frequency")
plt.savefig("lowest_against.png")

plt.clf()
plt.hist(highest_against, bins=20)
plt.xlabel("pitcher's AVG")
plt.xlim(0.1, 0.4)
plt.ylabel("frequency")
plt.savefig("highest_against.png")